### Imports

In [ ]:
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms, utils
from torch.utils.data import TensorDataset, DataLoader
import torch.backends.cudnn as cudnn
import matplotlib.pyplot as plt
import time
import h5py
from pathlib import Path
import os
import matplotlib.pylab as pl

from art.attacks.evasion import FastGradientMethod, CarliniL2Method, CarliniLInfMethod
from art.estimators.classification import PyTorchClassifier
from art.utils import load_cifar10

from _utils import train, test

%matplotlib inline
%config InlineBackend.figure_format='retina'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_cuda = True

### Data

In [ ]:
(x_train, y_train), (x_test, y_test), min_, max_ = load_cifar10()

x_train = np.swapaxes(x_train, 1, 3).astype(np.float32)
x_test = np.swapaxes(x_test, 1, 3).astype(np.float32)

train_dataset = TensorDataset(torch.Tensor(x_train), torch.Tensor(y_train))
train_dataloader = DataLoader(train_dataset, batch_size=128)

test_dataset = TensorDataset(torch.Tensor(x_test), torch.Tensor(y_test))
test_dataloader = DataLoader(test_dataset, batch_size=1000)
test_dataloader_single =  DataLoader(test_dataset, batch_size=1)

### Classifiers

In [ ]:
from ResNet18 import Bottleneck, BasicBlock

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)
    
    #     PRUNING
    activations = []
    mask = torch.zeros(100, 128, 16, 16)

    
    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)


    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out
    

    def forwardDetect(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        self.activations.append(out.view(out.size(0), -1))
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        self.activations.append(out)
        out = self.linear(out)
        return out
    
    def forwardMask(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)*self.mask
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out



def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])



# MASK NET FOR PRUNING!

class MaskResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(MaskResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)
    
    #     PRUNING
    mask = torch.ones(128, 128, 16, 16)

    
    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)


    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)*self.mask.to(device)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out
    
    

def MaskResNet18():
    return MaskResNet(BasicBlock, [2, 2, 2, 2])

### Load pre-trained models

In [ ]:
PATH = 'ResNet18.pth'

# Define what device we are using
print("CUDA Available: ",torch.cuda.is_available())
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

# Initialize models.
net = ResNet18().to(device)
maskNet = MaskResNet18().to(device)

# Load pre-trained model
net.load_state_dict(torch.load(PATH, map_location='cpu'))
maskNet.load_state_dict(torch.load(PATH, map_location='cpu'))


# Load loss and optimiser
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1,
                      momentum=0.9, weight_decay=5e-4)

# Make a classifier wrapper!
classifier = PyTorchClassifier(
    model=net,
    clip_values=(min_, max_),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(3, 32, 32),
    nb_classes=10,
)

maskClassifier = PyTorchClassifier(
    model=maskNet,
    clip_values=(min_, max_),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(3, 32, 32),
    nb_classes=10,
)

# Test model
predictions = classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {} %".format(accuracy * 100))

N = 128*30
predictions = maskClassifier.predict(x_test[:N])
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test[:N], axis=1)) / N
print("Accuracy on benign test examples: {} %".format(accuracy * 100))

### Pruning

In [ ]:
net.eval()
with torch.no_grad():
    for data, target in test_dataloader:
        output = net.forwardDetect(data.to(device))
        
activations = net.activations

In [ ]:
n = len(activations)
element0 = activations.pop()
act = torch.zeros(n,element0.size(0),element0.size(1))
act[0,:,:] = element0

for e in range(n-1):
    elementX = activations.pop()
    act[e+1,:,:] = elementX

mean_activations = torch.mean(act,dim=[0,1])
ma = mean_activations.numpy()
plt.hist(ma,80)
plt.title('Activations')
plt.show()

In [ ]:
# save different masks for different percentages

masks = dict()

for percentage in [0, 3, 6, 9]:
    s_ma, idx_ma = torch.sort(mean_activations)
    m = torch.ones(mean_activations.size()).to(device)
    nn = mean_activations.size(0)
    ind_r = round((percentage/10)*nn)
    m[idx_ma[:ind_r]] = 0.0
    n1 = m.sum()
    net.mask = m
    masks[percentage/10] = net.mask

### FGSM attack

In [ ]:
epsilons = [.001, .01, .1]

In [ ]:
fgsm_prune = dict()


# Run test for each epsilon and mask PG 
for (pg, mask) in masks.items():
    print('\n Pruned {}'.format(pg))
    accuracies = []
    results = dict()
    
#     if masking a Conv layer it will need reshaping
    mask_reshaped = torch.reshape(mask,(128, 16, 16)) 
    maskNet.mask = mask_reshaped
    test(maskNet)
    
    for e in epsilons:
        adv_crafter = FastGradientMethod(maskClassifier, eps=e)
        x_test_adv = adv_crafter.generate(x=x_test[:N])
        predictions = maskClassifier.predict(x_test_adv)
        accuracy = 100.*(np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test[:N], axis=1)) / N)
        accuracies.append(accuracy)
        print("Epsilon: {}   Test Accuracy = {}".format(e, accuracy))

    results['accuracies'] = accuracies
    results['epsilons'] = epsilons
    fgsm_prune[pg] = results

### C&W attack

In [ ]:
# Generate adversarial test examples
def CarliniL2(classifier, x_test, init_const=0.01):
    attack = CarliniL2Method(classifier=classifier,
                            confidence=0.0,
                            targeted=False,
                            learning_rate=0.01,
                            binary_search_steps=1,
                            initial_const=init_const,
                            batch_size=1)
    x_test_adv = attack.generate(x=x_test)
    return x_test_adv

def CarliniLInf(classifier, x_test, epsilon=0.005):
    attack = CarliniLInfMethod(classifier=classifier, 
                              confidence=0.0,
                              targeted=False, 
                              learning_rate=0.01,
                              eps=epsilon, 
                              batch_size=128)
    x_test_adv = attack.generate(x=x_test)
    return x_test_adv

In [ ]:
N = 100
x_test_adv = CarliniL2(classifier, x_test[:N])
predictions = classifier.predict(x_test_adv)
np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test[:N], axis=1)) / N

In [ ]:
# create adversarial samples on pruned models and attack pruned models 
n = 100
for (pg, mask) in masks.items():
    print('\n Pruned {}'.format(pg))
    accuracies = []
    results = dict()
    
#     if masking a Conv layer it will need reshaping
    mask_reshaped = torch.reshape(mask,(128, 16, 16)) 
    maskNet.mask = mask_reshaped
    
    x_test_adv_c2 = CarliniL2(maskClassifier, x_test[:n])
    predictions = maskClassifier.predict(x_test_adv_c2)
    acc = 100*(np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test[:n], axis=1)) / n)
    
    print(' Accuracy:', acc)

In [ ]:
N = 100
x_test_adv = CarliniLInf(classifier, x_test[:N])
predictions = classifier.predict(x_test_adv)
np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test[:N], axis=1)) / N

In [ ]:
# create adversarial samples on pruned models and attack pruned models 
n = 1000
for (pg, mask) in masks.items():
    print('\n Pruned {}'.format(pg))
    accuracies = []
    results = dict()
    
#     if masking a Conv layer it will need reshaping
    mask_reshaped = torch.reshape(mask,(256, 8, 8)) 
    maskNet.mask = mask_reshaped
    
    x_test_adv_c2 = CarliniLInf(maskClassifier, x_test[:n])
    predictions = maskClassifier.predict(x_test_adv_c2)
    acc = 100*(np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test[:n], axis=1)) / n)
    
    print(' Accuracy:', acc)